In [64]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F

In [2]:
class Error(Exception):
    def __init__(self, expr = None, msg = None):
        self.expr = expr
        self.msg = msg
class inputSmallerThanKernel(Error):
    def __init__():
        super(inputSmallerThanKernel, self).__init__()
class nodeDoesNotExist(Error):
    def __init__():
        super(nodeDoesNotExist, self).__init__()

In [79]:
class node(object):
    nodes = []
    def __init__(self, input_shape = (0, 0, 0), output_shape = (0, 0, 0)): # c, i1, i2
        node.nodes.append(self)
        self.no  = len(node.nodes)
        self.in_adj = []
        self.out_adj = []
        self.input_shape = input_shape
        self.output_shape = output_shape
        self.compatible = True
            
    def node_alright(self, curr_node):
        try:
            assert(issubclass(type(curr_node), node))
        except:
            raise Error('Not a node')
# Put this section in graph class
#         try:
#             assert(curr_node in graph_nodes)
#         except:
#             raise nodeDoesNotExist
    
    def determine_compatibility(self):
        for curr_node in self.in_adj:
            curr = (curr_node.output_shape == self.input_shape)
            self.compatible  = self.compatible and curr
            
        for curr_node in self.out_adj:
            curr = (curr_node.input_shape == self.output_shape)
            self.compatible = self.compatible and curr

    def describe_adj_list(self, in_adj, out_adj):
        for curr_node in in_adj + out_adj:
            try:
                node_alright(curr_node)
            except Exception as e:
                raise e
        self.in_adj = in_adj
        self.out_adj = out_adj

    def forward(self, x):
        pass
            
    def out_shape(self):
        pass
    
    def remove(self):
        ## needed in graph class
        pass
    
    
    def __repr__(self):
        return str(self.no) + " " + str(type(self))

In [80]:
class convolution_block(nn.Module, node):
    all_convs = []
    
    def __init__(self, in_h, in_w, in_channels, out_channels, kernel_size, padding = 0, stride = 1):
        try:
            assert(min(in_h, in_w) +2*padding > kernel_size)
        except:
            raise inputSmallerThanKernel
        super(convolution_block, self).__init__()
        node.__init__(self, (in_channels, in_h, in_w))
        convolution_block.all_convs.append(self)
        self.in_channels  = in_channels
        self.out_channels = out_channels 
        self.kernel_size = kernel_size
        self.stride = stride 
        self.padding = padding
        self.output_shape = self.out_shape()
        
        # NN Layers
        self.conv_layer = nn.Conv2d(self.in_channels, self.out_channels, self.kernel_size, self.stride, self.padding)
        self.batch_norm = nn.BatchNorm2d(self.out_channels)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.conv_layer(x)
        x = self.batch_norm(x)
        x = self.relu(x)
        return x

    def out_shape(self):
        c, h, w = self.input_shape
        C = self.out_channels
        H = (h + 2*self.padding - self.kernel_size)/self.stride + 1
        W = (w + 2*self.padding - self.kernel_size)/self.stride + 1
        return (C, H, W)
    
#     def determine_compatibility(self):
#         super(convolution_block, self).determine_compatibility()
#         self.compatible  = self.compatible and (len(self.in_adj) == 1)

    def describe_adj_list(self, in_adj, out_adj):
        try:
            assert(len(in_adj) == 1)
        except:
            raise Error('A convolution block can have only one in-edge')
        super(merge_node, self).describe_adj_list(in_adj, out_adj)

    
    def remove(self): 
        ### Remove from all_convs list
        pass

In [81]:
class max_pool_node(nn.Module, node):
    all_max_pools = []
    
    def __init__(self, in_h, in_w, in_channels, kernel_size, padding = 0, stride = 1):
        try:
            assert(min(in_h, in_w) +2*padding > kernel_size)
        except:
            raise inputSmallerThanKernel
        super(max_pool_node, self).__init__()    
        node.__init__(self, (in_channels, in_h, in_w))
        max_pool_node.all_max_pools.append(self)
        self.kernel_size = kernel_size
        self.stride = stride 
        self.padding = padding
        self.output_shape = self.out_shape()
        
        ## NN Layer
        self.max_pool_layer = nn.MaxPool2d(self.kernel_size, self.stride, self.padding)
        
    def forward(self, x):
        return self.max_pool_layer(x)
    
    def out_shape(self):
        c, h, w = self.input_shape
        C = c
        H = (h + 2*self.padding - self.kernel_size)/self.stride + 1
        W = (w + 2*self.padding - self.kernel_size)/self.stride + 1
        return (C, H, W)
    
#     def determine_compatibility(self):
#         super(max_pool_node, self).determine_compatibility()
#         self.compatible  = self.compatible and (len(self.in_adj) == 1)

    def describe_adj_list(self, in_adj, out_adj):
        try:
            assert(len(in_adj) == 1)
        except:
            raise Error('A max-pool block can have only one in-edge')
        super(merge_node, self).describe_adj_list(in_adj, out_adj)

    
    def remove(self):
        pass

In [82]:
### Is is it required to be to a derived class of nn.Module ?
class merge_node(nn.Module, node):
    all_merge_nodes = []
    
    def __init__(self, parents, child):
        super(merge_node, self).__init__()
        node.__init__(self)
        try:
            self.describe_adj_list([parents[0], parents[1]], [child])
        except Exception as e:
            raise e
        merge_node.all_merge_nodes.append(self)
        
    def describe_adj_list(self, in_adj, out_adj):
        try:
            assert(len(in_adj) == 2)
        except:
            raise Error('Parents must be exactly two')
        super(merge_node, self).describe_adj_list(in_adj, out_adj)
        
        
class add_node(merge_node):
    all_add_nodes = []
    
    def __init__(self, parents, child):
        super(add_node, self).__init__(parents, child)
        add_node.all_add_nodes.append(self)
        self.input_shape = self.in_adj[0].output_shape
        self.output_shape = self.out_shape()
    
    ### Does it allow to input paramteters ?
    def forward(self, x, y):
        return x+y ### Check if their data strcutre type supports this addition
        
    def out_shape(self):
        return self.input_shape
    
class concat_node(merge_node):
    # Define Later
    pass

class convex_merge_node(merge_node):
    # Define Later
    pass

In [83]:
class Network(object):
    def __init__(self):
        self.adj_mat = {}
        self.adj_list = {}
        self.nodes = set()
        self.int_to_node = {}
        self.conv_blocks = []
        self.max_pool_blocks = [] # Change naming conv maybe ?
        self.topsort = []
        
    def __init__(self, adj_list, int_to_node):
        try:
            assert(type(adj_list) == dict)
        except:
            raise Error('adj_list should be a dictionary')
        for _, li in adj_list.items():
            try:
                assert(type(li) == list)
                assert(len(li) == 2)
                assert(li[0] == list and li[1] == list)
            except:
                raise Error('Each mapping in adj_list should be to a two-dim list')
            try:
                assert(sorted(adj_list.keys()) == sorted(int_to_node.keys()))
            except:
                raise Error('Mismatch between nodes in adj_list and int_to_node')
        
        self.adj_list = adj_list
        self.adj_mat = self.get_adj_mat(self.adj_list)
        self.nodes = set(int_to_node.keys())
        self.int_to_node = int_to_node
        self.conv_blocks, self.max_pool_blocks = self.get_conv_and_max_pool_blocks()
        self.topsort = self.topsorting()
        
    def __init__(self, random_init):
        if random_init:
            # Do random network construction
            pass
        else:
            self.__init__()
        
    def get_adj_mat(self, adj_list):
        adj_mat = {}
        nodes = adj_list.keys()
        for x in nodes:
            adj_mat[x] = {}
            for y in nodes:
                adj_mat[x][y] = 0
        for node, li in adj_list.items():
            for child in li:
                adj_mat[node][child] = 1
        return adj_mat
    
    def get_conv_and_max_pool_blocks(self):
        conv_blocks = []
        max_pool_blocks = []
        for x in self.nodes:
            if isinstance(self.int_to_node[x], convolution_block):
                conv_blocks.append(x)
            elif isinstance(self.int_to_node[x], max_pool_node):
                max_pool_blocks.append(x)
        return (conv_blocks, max_pool_blocks)
    
    def topsorting(self):
        # level problem
        topsort = []
        import Queue
        in_deg = {}
        q = queue.Queue()
        for node in self.nodes:
            val = len(self.int_to_node[node].in_adj)
            if val == 0:
                q.put(node)
            in_deg[node] = val
            
        while not q.empty():
            curr_node = q.get()
            topsort.append(curr_node)
            for child in self.adj_list[curr_node]:
                in_deg[child] -= 1
                if in_deg[child] == 0:
                    q.put(child)
        return topsort
    
    
    def morphism(self):
        import random
        actions = {'deepen': self.deepen_morph, 
                   'widen': self.widen_morph, 
                   'skip-connection': self.skip_morph }
        choice = random.choice(actions)
        actions[choice]()
    
    def deepen_morph(self):
        deepen_conv_block = self.int_to_node[random.choice(self.conv_blocks)]
        kernel_size = random.choice([3, 5])
        in_channels, in_h, in_w = deepen_conv_block.output_shape
        out_channels = in_channels
        identity_conv_block = convolution_block(in_h, in_w, in_channels, out_channels, kernel_size, (kernel_size-1)/2)
        ## make connections 
        identity_conv_block.describe_adj_list([deepen_conv_block], deepen_conv_block.out_adj)
        deepen_conv_block.describe_adj_list(deepen_conv_block.in_adj, [identity_conv_block])
        weights = identity_conv_block.conv_layer.weight.data
        for channel in range(out_channels):
            for i in range(in_channels):
                for j in range(kernel_size):
                    for k in range(kernel_size):
                        weights[channel][i][j][k] = (channel == i) and (j == k) and j == (kernel_size)/2 
                            
        ## weights

        

    def widen_morph(self):
        pass
    
    def skip_morph(self):
        pass
    
        
    
    

In [77]:
import random
random.choice(['deepen', 'widen', 'skip-connection'])

TypeError: 'set' object does not support indexing